In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['P0', 'P1', 'P2', 'P3', 'P4']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
#!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
#!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    clust.add_input_data("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv", "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    #!rm autoclass.r-params
    #!cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/label.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['label'].value_counts().max()/x.shape[0])
            listApp.append(x['label'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(list(afterall.items()), index= range(len(subfeatures)))
round1.info()
round1.head(1)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/Hp1.csv', index = True, header=True)

2020-04-27 16:22:54 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P0.tsv' as 'real location' with error 0.01
2020-04-27 16:22:54 INFO     Detected encoding: ascii
2020-04-27 16:22:54 INFO     Found 3200 rows and 2 columns
2020-04-27 16:22:54 DEBUG    Checking column names
2020-04-27 16:22:54 DEBUG    Index name 'index'
2020-04-27 16:22:54 DEBUG    Column name 'P0'
2020-04-27 16:22:54 INFO     Checking data format
2020-04-27 16:22:54 INFO     Column 'P0'
2020-04-27 16:22:54 INFO     count    3.200000e+03
2020-04-27 16:22:54 INFO     mean    -7.765439e+03
2020-04-27 16:22:54 INFO     std      1.036135e+06
2020-04-27 16:22:54 INFO     min     -1.783918e+06
2020-04-27 16:22:54 INFO     50%      6.245000e+03
2020-04-27 16:22:54 INFO     max      1.786032e+06
2020-04-27 16:22:54 INFO     ---
2020-04-27 16:22:54 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv' as 'real location' with 

Time: 822 sec.

2020-04-27 16:36:37 INFO     Extracting autoclass results
2020-04-27 16:36:37 INFO     Found 3200 cases classified in 57 classes
2020-04-27 16:36:42 INFO     Aggregating input data
2020-04-27 16:36:42 INFO     Writing classes + probabilities .tsv file
2020-04-27 16:36:42 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P1.tsv' as 'real location' with error 0.01
2020-04-27 16:36:42 INFO     Detected encoding: ascii
2020-04-27 16:36:42 INFO     Found 3200 rows and 2 columns
2020-04-27 16:36:42 DEBUG    Checking column names
2020-04-27 16:36:42 DEBUG    Index name 'index'
2020-04-27 16:36:42 DEBUG    Column name 'P1'
2020-04-27 16:36:42 INFO     Checking data format
2020-04-27 16:36:42 INFO     Column 'P1'
2020-04-27 16:36:42 INFO     count    3200.000000
2020-04-27 16:36:42 INFO     mean       -0.300688
2020-04-27 16:36:42 INFO     std         3.208397
2020-04-27 16:36:42 INFO     min        -3.908736
2020-04-27 16:36:42 INFO     50%      

[0.09836065573770492, 0.1056338028169014, 0.08108108108108109, 0.08074534161490683, 0.08620689655172414, 0.12037037037037036, 0.2857142857142857, 0.125, 0.07964601769911504, 0.11224489795918367, 0.09302325581395349, 0.1839080459770115, 0.13095238095238096, 0.11, 0.22666666666666666, 0.0945945945945946, 0.2, 0.13924050632911392, 0.5263157894736842, 0.11842105263157894, 0.2608695652173913, 1.0, 0.3050847457627119, 0.08, 0.2909090909090909, 0.11320754716981132, 0.9787234042553191, 0.2, 0.25, 0.3541666666666667, 0.22727272727272727, 0.2553191489361702, 0.375, 0.175, 0.25, 0.09523809523809523, 0.16279069767441862, 0.10526315789473684, 0.17142857142857143, 0.2647058823529412, 0.32432432432432434, 0.1388888888888889, 0.11428571428571428, 0.2903225806451613, 0.13793103448275862, 0.21428571428571427, 0.11538461538461539, 0.25, 0.21739130434782608, 0.13333333333333333, 0.2857142857142857, 0.75, 0.11764705882352941, 0.16666666666666666, 0.2222222222222222, 0.3333333333333333, 0.5]
['EBAY', 'APPLE

2020-04-27 16:36:43 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:36:43 INFO     Writing run file
2020-04-27 16:36:43 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:36:43 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:36:43 INFO     Running clustering...


...

[0.07894736842105263, 0.08385093167701864, 0.09154929577464789, 0.08241758241758242, 0.19391634980988592, 0.1076923076923077, 0.1111111111111111, 0.1782178217821782, 0.08943089430894309, 0.1206896551724138, 1.0, 0.25, 0.14130434782608695, 0.18840579710144928, 0.19718309859154928, 0.19148936170212766, 0.16666666666666666, 0.34210526315789475, 0.09523809523809523, 0.171875, 0.9565217391304348, 0.25, 0.1276595744680851, 0.3333333333333333]
['APPLE_ICLOUD', 'APPLE_ICLOUD', 'HTTP_PROXY', 'YOUTUBE', 'CONTENT_FLASH', 'HTTP_CONNECT', 'DROPBOX', 'WINDOWS_UPDATE', 'WINDOWS_UPDATE', 'FACEBOOK', 'IP_ICMP', 'OFFICE_365', 'FACEBOOK', 'INSTAGRAM', 'HTTP', 'DNS', 'APPLE_ITUNES', 'AMAZON', 'SPOTIFY', 'CONTENT_FLASH', 'DNS', 'HTTP', 'HTTP_PROXY', 'OFFICE_365']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
0.23123356647428603


2020-04-27 17:01:52 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 17:01:52 INFO     Writing run file
2020-04-27 17:01:52 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 17:01:52 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 17:01:52 INFO     Running clustering...


Time: 486 sec.

2020-04-27 17:09:59 INFO     Extracting autoclass results
2020-04-27 17:09:59 INFO     Found 3200 cases classified in 19 classes
2020-04-27 17:10:02 INFO     Aggregating input data
2020-04-27 17:10:02 INFO     Writing classes + probabilities .tsv file


[0.06349206349206349, 0.07293666026871401, 0.09090909090909091, 0.09349593495934959, 0.0861244019138756, 0.14566929133858267, 0.10135135135135136, 0.3466666666666667, 0.13008130081300814, 0.10714285714285714, 1.0, 0.16455696202531644, 0.6756756756756757, 0.3404255319148936, 0.390625, 0.11904761904761904, 0.09375, 0.19047619047619047, 0.14285714285714285]
['MS_ONE_DRIVE', 'HTTP_CONNECT', 'HTTP_PROXY', 'APPLE_ICLOUD', 'WINDOWS_UPDATE', 'AMAZON', 'CLOUDFLARE', 'CONTENT_FLASH', 'APPLE', 'WINDOWS_UPDATE', 'IP_ICMP', 'HTTP', 'DNS', 'OFFICE_365', 'OFFICE_365', 'GOOGLE', 'NETFLIX', 'MSN', 'INSTAGRAM']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
0.22922546004486305
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5 non-null      object 
 1   1       5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes
